In [1]:
#Imports
import sys
sys.path.append('../python/')
import NGC5533_functions as nf
import noordermeer as noord
import fitting_NGC5533 as fitting
import dataPython as dp

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interactive, fixed, FloatSlider, HBox, Layout, Button, Label, Output, VBox

from IPython.display import display, clear_output
from IPython.display import Javascript

import scipy.stats as stats
import warnings
warnings.filterwarnings("ignore")  #ignore warnings

In [2]:
#TRACING:**************************************
#data points:
data = dp.getXYdata_wXYerr('../data/final/nord-120kpc-datapoints.txt')
r_dat = np.asarray(data['xx'])
v_dat = np.asarray(data['yy'])
v_err0 = np.asarray(data['ex'])
v_err1 = np.asarray(data['ey'])

In [3]:
# Define components
M = fitting.f_M
bpref = fitting.f_c
dpref = fitting.f_pref
rc = fitting.f_rc
rho00 = fitting.f_hrho00
gpref = fitting.f_gpref
def blackhole(r):
    return nf.bh_v(r,M,load=False)

def bulge(r):
    return bpref*nf.b_v(r,load=True)

def disk(r):
    return dpref*nf.d_thief(r)

def halo(r,rc,rho00):
    return nf.h_v(r,rc,rho00,load=False)

def gas(r):
    return gpref*nf.g_thief(r)
def totalcurve(r,rc,rho00):
    total = np.sqrt(blackhole(r)**2 
                    + bulge(r)**2 
                    + disk(r)**2
                    + halo(r,rc,rho00)**2
                    + gas(r)**2)
    return total

In [4]:
#best fitted prefactor values for each component, to be used as default (initial) values for widget sliders
best_rc = fitting.f_rc
best_rho00 = fitting.f_hrho00

import scipy.integrate as si             # for integration

# Equation for isothermal density
def density_iso(r,rho00,rc):
    density = rho00 * (1 + (r/rc)**2)**(-1)
    return density

# Equation for mass as a function of radius
def mass_function(r,rho00,rc):
    mass = 4 * np.pi * density_iso(r,rho00,rc) * r**2
    return mass

# Integrate to calculate total mass enclosed
Mass = lambda r,rho00,rc: si.quad(mass_function, 0, r, args=(rho00,rc))[0]
radius = 100   # in kpc, approximation of infinity; the radius within which the total mass of the halo is being calculated

'''
##finding mass?
rr=max(r_dat) #kpc
h=8.9/2 #kpc; is this the thickness of the galaxy???
volume=np.pi*2*rr**2*h
def Mass(rc,rho00):
    density=rho00*(1+(rr/rc)**2)**-1
    Mass = volume*density
    return Mass
'''
print(Mass(1,1,1))

2.6967662132698145


In [5]:
# Define plotting function
def f(rc,rho00):
    
    # Define r
    r = np.linspace(0.1,13,1000)
    
    # Plot
    plt.figure(figsize=(12,8))
    plt.xlim(0,13)
    plt.ylim(0,360)
    
    plt.errorbar(r_dat,v_dat,yerr=v_err1,fmt='bo',label='Data')
    plt.plot(r,blackhole(r),label=("Black Hole"),color='black')
    plt.plot(r,bulge(r),label=("Bulge"),color='orange')
    plt.plot(r,disk(r),label=("Disk"),color='purple')
    plt.plot(r,halo(r,rc,rho00),label=("Halo"),color='green')
    plt.plot(r,gas(r),label=("Gas"),color='blue')
    plt.plot(r,totalcurve(r,rc,rho00),label=("Total Curve"),color='red')
    plt.plot(r,(blackhole(r)**2+bulge(r)**2+disk(r)**2+gas(r)**2)**(1/2),label=("All Luminous Matter"),linestyle='--')
    plt.fill_between(r,
                     noord.greyb_bottom(r),noord.greyb_top(r),
                     color='#dddddd')
    plt.title("Interactive Rotation Curve - Galaxy: NGC 5533")
    plt.xlabel("Radius (kpc)")
    plt.ylabel("Velocity (km/s)")
    
    # Chi squared and reduced chi squared
    # Residuals
    r = np.linspace(0.1,100,69)
    residuals = v_dat - totalcurve(r_dat,rc,rho00)
    residuals[0] = v_dat[0] #set totalcurve to 0 at 0 - currently going to infinity which results in fitting issues.
    # Determining errors
    errors = v_err1 #np.sqrt(v_err1**2 + noord.band**2) #inclination uncertainty shouldn't apply to this galaxy as we don't have one given.
    # Chi squared
    chisquared = np.sum(residuals**2/errors**2)
    #chisquared = stats.chisquare(v_dat,totalcurve(r,M,bpref,dpref,rc,rho00,gpref))
    reducedchisquared = chisquared * (1/(len(r_dat)-6))
    
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(10.15,325,r"$\chi^2$: {:.5f}".format(chisquared)+'\n'+r"Reduced: {:.5f}".format(reducedchisquared),bbox=props)
    #plt.text(80,150,,bbox=props)
    TotalMass = Mass(radius,rho00,rc)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(6,325,r"Total Dark Matter Mass: {:.2e}".format(TotalMass)+'$M_{\odot}$',bbox=props)
    plt.legend(loc='lower right')
    plt.annotate('Data source: E. Noordermeer. The rotation curves of flattened Sérsic bulges. MNRAS,385(3):1359–1364, Apr 2008',
            xy=(0, 0), xytext=(0,5),
            xycoords=('axes fraction', 'figure fraction'),
            textcoords='offset points',
            size=10, ha='left', va='bottom')
    
    plt.show()
    

In [6]:
# Appearance
style = {'description_width': 'initial'}
layout = {'width':'600px'}

# Define slides


rc = FloatSlider(min=0.1, max=5, step=0.1, 
                    value=0.01, 
                    description='Halo Core Radius [kpc]', 
                    readout_format='.2f', 
                    orientation='horizontal', 
                    style=style, layout=layout)
rho00 = FloatSlider(min=0, max=1e9, step=1e7, 
                    value=.1, 
                    description=r'Halo Surface Density [$M_{\odot} / pc^3$]', 
                    readout_format='.2e', 
                    orientation='horizontal', 
                    style=style, layout=layout)

# Interactive widget
def interactive_plot(f):
    interact = interactive(f,
                               rc = rc,
                               rho00 = rho00,
                               continuous_update=False)
    return interact

# Button to revert back to Best Fit
button = Button(
    description="Best Fit",
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='check')
out = Output()

def on_button_clicked(_):
    #display(Javascript('IPython.notebook.execute_cells_below()'))
    rc.value=best_rc
    rho00.value = best_rho00

button.on_click(on_button_clicked)


## What do rotation curves look like without dark matter?
In this activity, you can visualize how important dark matter is to accurately describing observed data (marked in blue points with error bars below)

So how much mass does a dark matter halo need to contain (i.e. how much dark matter is in a given galaxy) to account for our observations?

In [7]:
# displaying button and its output together
VBox([button,out,interactive_plot(f)])

## Slider Key


The halo surface density behaves as a prefactor for the **dark matter "halo"** in and around the galaxy. 
Its units are solar masses per cubic parsec (and in this fit is on the scale of hundreds of millions).
This represents how much dark matter we think there is overall. 

Play around with the halo parameters and see how incorporating a dark matter component allows us to more accurately match our observations.

**Rembember, a reduced $\chi^2$ close to 1 is generally considered a good fit. So  reduced $\chi^2$ far higher or lower than 1 suggests a poor theoretical model (that is, the model probably doesn't accurately describe the real universe).**

*key*

kpc = kiloparsec, equal to 3.26 light years, or $1.917 \times 10^{16}$ miles <br>
km/s = kilometers per second (this is a velocity) <br>
$M_{\odot}$ = solar masses (X number of masses equal to our sun), approximately equal to $1.989 \times 10^{30}$ kilograms <br>
$M_{\odot} / pc^3$ = solar masses per parsec cubed (this is a 3D density) <br>
